In [12]:
#imports
import numpy as np 
import pandas as pd 
import os
import cv2
from tqdm import tqdm

Extract the frames from video

In [13]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Function to extract frames from a video
def extract_frames(video_path, output_folder, frame_interval=10):
    cap = cv2.VideoCapture(video_path)
    
    # Get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Video FPS: {fps}")
    
    frame_count = 0
    last_frame = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames if the interval hasn't been reached
        if frame_count % frame_interval == 0:
            if last_frame is not None:
                # Check if the frame is identical to the previous one
                if np.array_equal(last_frame, frame):
                    continue  # Skip this frame

            # Save the frame
            frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            last_frame = frame  # Update the last frame

        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames from {video_path} into {output_folder}.")

# Paths for the video and output frames folder
video_file_path = '/kaggle/input/road-srilanka-1/input_video.mp4'
output_base_folder = '/kaggle/working/frames'
video_name = os.path.splitext(os.path.basename(video_file_path))[0]
video_output_folder = os.path.join(output_base_folder, video_name)

os.makedirs(video_output_folder, exist_ok=True)

# Extract frames with a frame interval of 10
extract_frames(video_file_path, video_output_folder, frame_interval=10)

/kaggle/input/road-srilanka-1/input_video.mp4
Video FPS: 25.0
Extracted 294 frames from /kaggle/input/road-srilanka-1/input_video.mp4 into /kaggle/working/frames/input_video.


Preprocessing the Extracted frames

In [14]:

# Paths
input_frames_folder = '/kaggle/working/frames/input_video'
preprocessed_folder = '/kaggle/working/preprocessed_frames'

# Preprocessing parameters
TARGET_SIZE = (640, 640)  
NORMALIZE = True          
JPEG_QUALITY = 95       

# Ensure the output folder exists
os.makedirs(preprocessed_folder, exist_ok=True)

def preprocess_image(image_path, output_folder, target_size=(640, 640), normalize=True):
    image = cv2.imread(image_path)
    if image is None:
        return

    # Resize the image
    image_resized = cv2.resize(image, target_size, interpolation=cv2.INTER_CUBIC)

    # Normalize the image
    if normalize:
        image_resized = image_resized / 255.0

    # Save the preprocessed image
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    
    # Save as JPEG with specified quality
    cv2.imwrite(output_path, (image_resized * 255).astype(np.uint8), [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
    
    return output_path

def preprocess_frames(input_folder, output_folder, target_size=(640, 640), normalize=True):
    frame_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]
    
    if not frame_files:
        return

    for frame_file in tqdm(frame_files, desc="Preprocessing frames"):
        frame_path = os.path.join(input_folder, frame_file)
        preprocess_image(frame_path, output_folder, target_size, normalize)

# Run the preprocessing
preprocess_frames(input_frames_folder, preprocessed_folder, target_size=TARGET_SIZE, normalize=NORMALIZE)


Preprocessing frames: 100%|██████████| 176/176 [00:02<00:00, 59.84it/s]
